### Agents

##### Question 2 - Personalized News Curator

In [103]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.agents import Tool, load_tools, create_react_agent, AgentExecutor, tool
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_community.document_loaders import WebBaseLoader
import os
import requests

In [104]:
gemini = ChatGoogleGenerativeAI(model = "gemini-pro")
openai = ChatOpenAI(openai_api_key=os.getenv('OPENAI_API_KEY_PERSONAL'))
interests = []

In [105]:
i = int(input("how many topics would you like to enter"))
for _ in range(i):
    userInterest = input("Please type your topics of interest separated with a comma")
    interests.append(userInterest)

In [106]:
print(interests)

['sports', 'entertainment', 'politics']


In [107]:
user_content_list = []
for i in interests:
    api_key = os.getenv("newsapi")
    url = 'https://newsapi.org/v2/everything'

    params = {
        'q': i,
        'from': '2024-02-01',
        'to': '2024-02-13',
        'sortBy': 'popular',
        'apiKey': api_key, 
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        articles = data.get('articles', [])
        count = 0
        for article in articles:
            print(f"Title: {article['title']}")
            print(f"Author: {article['author']}")
            print(f"Description: {article['description']}")
            print(f"URL: {article['url']}")
            print(f"Published At: {article['publishedAt']}")
            print("-----")
            
            user_content_list.append(article['url'])
            
            count += 1
            if count >= 4:
                break
    else:
        print(f"Error: {response.status_code}, {response.text}")

    

Title: ESPN, Fox, and Warner Bros. Discovery are launching a streaming service just for sports this fall
Author: Pranav Dixit
Description: Three of the biggest sports TV companies in the US — ESPN, Fox, and Warner Bros. Discovery — will launch a streaming sports service in the fall of 2024, the companies said in a joint statement on Tuesday. It will stream sporting events from networks that all …
URL: https://consent.yahoo.com/v2/collectConsent?sessionId=1_cc-session_cea6a5eb-1a79-4664-97fc-06941e8ca9e6
Published At: 2024-02-06T22:50:50Z
-----
Title: Chrysler Halcyon concept is a reminder the company can be more than just minivans
Author: Andrew J. Hawkins
Description: Chrysler revealed its Halcyon concept, a stunning four-door sports car with electric and autonomous capabilities. And more importantly, it’s not a minivan.
URL: https://www.theverge.com/2024/2/13/24067683/chrysler-halcyon-concept-ev-autonomous-specs-photos
Published At: 2024-02-13T10:01:00Z
-----
Title: ESPN, Fox, and Wa

In [108]:
summary_list = []
for i in (user_content_list):
    loader = WebBaseLoader(i)
    loaded_urls = loader.load()
    content = loaded_urls[0].page_content
    summary_list.append(openai.invoke(f"summarize the content in 2-3 sentences: \n{content}"))
    

In [109]:
# checking the summary_list 
summary_list

[AIMessage(content='The content seems to be a technical error message related to Yahoo. It instructs the user to try again later.'),
 AIMessage(content="Chrysler's Halcyon concept showcases the company's vision for the future beyond minivans, with a sleek, high-tech roadster featuring electric powertrain and autonomous capabilities. The concept highlights Chrysler's commitment to advanced technology and sustainability, with features like recycled materials, AI voice assistants, and predictive navigation. While the Halcyon is not yet production-ready, it serves as a demonstration of Chrysler's future direction towards electric vehicles and cutting-edge technologies."),
 AIMessage(content='ESPN, Fox, and Warner Bros. are collaborating to launch a sports streaming app that will include a wide range of sports leagues and networks. This new service is set to offer a comprehensive selection of sports programming, potentially addressing the fragmented nature of sports streaming options curren

In [110]:
print(summary_list[0].content)

The content seems to be a technical error message related to Yahoo. It instructs the user to try again later.


In [111]:
suggested_urls = []
@tool
def suggest_articles(text):
    '''Used to suggest 9 new articles to the user based on the list of summaries provided'''
    for i in interests:
        api_key = os.getenv("newsapi")
        url = 'https://newsapi.org/v2/everything'

        params = {
            'q': i,
            'from': '2024-02-14',
            'to': '2024-02-25',
            'sortBy': 'popular',
            'apiKey': api_key, 
        }

        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            articles = data.get('articles', [])
            count = 0
            for article in articles:
                print(f"Title: {article['title']}")
                print(f"Author: {article['author']}")
                print(f"Description: {article['description']}")
                print(f"URL: {article['url']}")
                print(f"Published At: {article['publishedAt']}")
                print("-----")
                
                suggested_urls.append(article['url'])
                
                count += 1
                if count >= 3:
                    break
        else:
            print(f"Error: {response.status_code}, {response.text}")   

tools = [suggest_articles] 

In [112]:
predefinedPrompt = hub.pull("hwchase17/react") 
agent = create_react_agent(llm = openai, tools = tools, prompt = predefinedPrompt)
agent_executor = AgentExecutor(agent=agent, tools = tools, verbose = True)
result = agent_executor.invoke({"input": [summary_list]})




> Entering new AgentExecutor chain...
There are multiple summaries included in the input question.
Action: suggest_articles
Action Input: The list of summaries provided in the input questionTitle: Max is the first streaming service to offer Dolby Vision for live sports
Author: Steve Dent
Description: As part of its NBA All-Star coverage press release, Warner Bros. Discovery announced that it'll stream live sports with Dolby Vision for the first time ever. The feature will be available to Max subscribers with supported Dolby Vision TVs "via the B/R [Bleach…
URL: https://consent.yahoo.com/v2/collectConsent?sessionId=1_cc-session_fb11e2fd-f2de-450f-818f-e6aeda5a764d
Published At: 2024-02-14T10:15:15Z
-----
Title: FuboTV accuses Disney, Fox and Warner Bros. of antitrust practices over joint streaming service
Author: Mariella Moon
Description: FuboTV, a streaming platform dedicated to live sports, has filed an antitrust lawsuit against Disney, Fox and Warner Bros. Discovery, accusing the 

In [113]:
print(result["output"])

The 9 new articles have been suggested to the user.


In [114]:
# Checking the recommended URLs

print(suggested_urls)

['https://consent.yahoo.com/v2/collectConsent?sessionId=1_cc-session_fb11e2fd-f2de-450f-818f-e6aeda5a764d', 'https://consent.yahoo.com/v2/collectConsent?sessionId=1_cc-session_03c89ec0-3401-4e53-8be7-8d8d08634c6c', 'https://www.wired.com/story/apple-vision-pro-crack-in-front-screen/', 'https://www.theverge.com/2024/2/16/24058318/apple-vision-pro-sharing-difficulties', 'https://www.engadget.com/sony-wants-its-playstation-exclusives-to-come-to-pc-earlier-212001939.html', 'https://www.engadget.com/a-piracy-app-outranked-netflix-on-the-app-store-before-apple-pulled-it-132013246.html', 'https://www.businessinsider.com/house-republicans-performance-politics-mayorkas-vote-2024-2', 'https://www.businessinsider.com/moving-from-florida-to-virginia-pros-cons-cape-coral-weather-2024-2', 'https://www.businessinsider.com/george-santos-pilip-new-york-election-loss-standards-suozzi-2024-2', 'https://consent.yahoo.com/v2/collectConsent?sessionId=1_cc-session_fb11e2fd-f2de-450f-818f-e6aeda5a764d', 'http

In [115]:
#  Proactive Suggestion Code 

from time import sleep
from plyer import notification

for link in suggested_urls:
    notification.notify(
        title='Latest News',
        message=link,
        timeout=10
    )
    sleep(5)

<<< End Of Code >>>